# Basic Copar Id Test (done for André), single years, no clustering

In [ ]:
import sys
import os

# Add parent directory to sys.path
sys.path.append(os.path.abspath(".."))


import numpy as np
import os
import getdata
import sortdata
import plotting

# ---- Configuration ----
BASE_DIR = os.path.join("output", "results_simulation_quarterly_10cm_with_id")
os.makedirs(BASE_DIR, exist_ok=True)
getdata.clear_directory(BASE_DIR)

YEAR_RANGES = [np.array([y]) for y in range(2019, 2024)]
SEEDS = [1]
META_FILE = getdata.read_metadata_file("../input/geogto.dat")

# ---- Main loop ----
for years in YEAR_RANGES:
    print(f"Processing years: {years}")
    # Prepare output directory for this year range
    year_label = years[0]
    out_dir = os.path.join(BASE_DIR, f"results_{year_label}")
    os.makedirs(out_dir, exist_ok=True)
    getdata.clear_directory(out_dir)

    # Step 1: Build file map for each (year, seed)
    monthly_files = {}  # (year, seed) -> [GEO_crs, GEO_det, GTO_crs, GTO_det, FOL_crs, FOL_det]
    for seed in SEEDS:
        for year in years:
            files = []
            for orbit in ("geo", "gto", "fol"):
                crs_files, det_files = getdata.find_monthly_files("../input", year, orbit, seed)
                files.extend([crs_files, det_files])
            monthly_files[(year, seed)] = files

    # Step 2: Load and stack CRS/DET arrays per seed
    crs_all = []  # list of (GEO, GTO, FOL) tuples per seed
    det_all = []
    for seed in SEEDS:
        crs_per_orbit = {orbit: [] for orbit in ("geo", "gto", "fol")}
        det_per_orbit = {orbit: [] for orbit in ("geo", "gto", "fol")}

        for year in years:
            files = monthly_files[(year, seed)]
            # files is [GEO_crs, GEO_det, GTO_crs, GTO_det, FOL_crs, FOL_det]
            for orbit, (crs_list, det_list) in zip(("geo","gto","fol"),
                                                  zip(files[0::2], files[1::2])):
                if not crs_list or not det_list:
                    print(f"Warning: no files for {year}, {orbit}, seed {seed}")
                    continue

                # Extend and horizontally stack
                crs_arrays = [getdata.array_extender(f) for f in crs_list]
                det_arrays = [getdata.array_extender(f) for f in det_list]
                crs_per_orbit[orbit].append(np.hstack(crs_arrays) if crs_arrays else np.empty((0,)))
                det_per_orbit[orbit].append(np.hstack(det_arrays) if det_arrays else np.empty((0,)))

        # Final per-seed stacks
        crs_all.append(
            tuple(np.hstack(crs_per_orbit[o]) if crs_per_orbit[o] else np.empty((0,))
                  for o in ("geo","gto","fol"))
        )
        det_all.append(
            tuple(np.hstack(det_per_orbit[o]) if det_per_orbit[o] else np.empty((0,))
                  for o in ("geo","gto","fol"))
        )

    # Step 3: Concatenate across seeds for each orbit
    all_GEO_crs, all_GTO_crs, all_FOL_crs = map(
        lambda idx: np.hstack([seed_tuple[idx] for seed_tuple in crs_all]),
        range(3)
    )
    all_GEO_det, all_GTO_det, all_FOL_det = map(
        lambda idx: np.hstack([seed_tuple[idx] for seed_tuple in det_all]),
        range(3)
    )

    # Step 4: Apply filtering, sorting and plotting
    getdata.data_monthly_one_seed_with_id(
        years,
        out_dir,
        SEEDS,
        monthly_files,
        META_FILE
    )

    # (Optional) Downstream combined sort of detections
    combined_det = np.hstack([arr for arr in (all_GEO_det, all_GTO_det, all_FOL_det) if arr.size])
    print(f"Combined detections shape: {combined_det.shape}")
    TLE, frag, rest = sortdata.data_sorter(combined_det, 8, 10, 9, 20, source_index=3, size_index=1)
    print(f"TLE: {TLE.shape}, frag: {frag.shape}, rest: {rest.shape}")
    # Merge fragment+rest if desired
    combined_filtered = frag if rest.size == 0 else rest if frag.size == 0 else np.vstack([frag, rest])
    print(f"Filtered combined shape: {combined_filtered.shape}")


Processing years: [2019]
Direct filenames ['../input\\stat_Master_19_geo_s1_02.crs', '../input\\stat_Master_19_gto_s1_02.crs', '../input\\stat_Master_19_fol_s1_02.crs', '../input\\stat_Master_19_geo_s1_02.det', '../input\\stat_Master_19_gto_s1_02.det', '../input\\stat_Master_19_fol_s1_02.det']
Direct filenames ['../input\\stat_Master_19_geo_s1_05.crs', '../input\\stat_Master_19_gto_s1_05.crs', '../input\\stat_Master_19_fol_s1_05.crs', '../input\\stat_Master_19_geo_s1_05.det', '../input\\stat_Master_19_gto_s1_05.det', '../input\\stat_Master_19_fol_s1_05.det']
Direct filenames ['../input\\stat_Master_19_geo_s1_11.crs', '../input\\stat_Master_19_gto_s1_11.crs', '../input\\stat_Master_19_fol_s1_11.crs', '../input\\stat_Master_19_geo_s1_11.det', '../input\\stat_Master_19_gto_s1_11.det', '../input\\stat_Master_19_fol_s1_11.det']
here {287: '1965-108A', 639: '1967-001D', 93: '1976-105A', 100: '1977-068A', 97: '1977-027A', 101: '1977-092A', 98: '1977-047A', 110: '1979-077A', 461: '1979-101A', 

# Apply COSPAR ID coloring to data used for clustering

In [20]:
import cluster_data_cospar
from cluster_data_cospar import normalize_data
import getdata  # Assuming prepare_data_for_clustering is in getdata
import numpy as np
import pandas as pd
import os
import cluster_plotter
import seaborn as sns
import matplotlib.pyplot as plt

# --- Setup ---
year_range = [2005, 2006, 2007, 2008]
#binned_data = cluster_data_cospar.bin_data_for_clustering(data_set, print_res=False)

plot_dir = r"Images\tasks_10072025_cospar_pairplots"
os.makedirs(plot_dir, exist_ok=True)
cluster_plotter.clear_directory(plot_dir)

top_n = 20

feature_names = [
    "Eccentricity e", "Semi major axis [km]", "Inclination [°]",
    "RAAN [°]", "Perigee [°]", "True latitude [°]",
    "Mean Motion [rev per day]", "Magnitude [mag]", "Diameter [m]"
]


print(f"\nProcessing Year Range: {year_range}")

orbits = ["geo", "gto", "fol"]
seed = 1
filenames = []

for year in year_range:
    year = str(year)[2:]
    for orbit in orbits:
        if int(year) > 21: 
            crs_files, det_files = getdata.find_monthly_files("../input/", year, orbit, seed)
        if year == "07":
            det_files = f"../input/stat_Master_{year}_{orbit}_janpo.det"
        else: 
            det_files = f"../input/stat_Master_{year}_{orbit}_s{seed}_janpo.det"
                    
        filenames.append(det_files)  


matched, unmatched = cluster_data_cospar.prepare_data_for_clustering(filenames)

# Combine features
matched_X = np.column_stack([
    matched.ecc, matched.sem_maj, matched.inc, matched.raan,
    matched.perigee, matched.true_lat, matched.mean_motion,
    matched.mag_obj, matched.diameter
])

unmatched_X = np.column_stack([
    unmatched.ecc, unmatched.sem_maj, unmatched.inc, unmatched.raan,
    unmatched.perigee, unmatched.true_lat, unmatched.mean_motion,
    unmatched.mag_obj, unmatched.diameter
])

X = np.vstack([matched_X, unmatched_X])

# Combine COSPAR_IDs
COSPAR_ID = np.concatenate([
    matched.cospar_id.astype(str),
    unmatched.cospar_id.astype(str)
])

# Sanity check
if len(COSPAR_ID) != X.shape[0]:
    raise ValueError(f"Length of COSPAR IDs ({len(COSPAR_ID)}) does not match data rows ({X.shape[0]})")

# Normalize if needed
normalized_data, data_min, data_max = normalize_data(X)

# Make DataFrame
dfp = pd.DataFrame(X, columns=feature_names)
dfp['COSPAR_ID'] = COSPAR_ID

# Count COSPAR occurrences, select top N
cospar_counts = dfp[dfp['COSPAR_ID'] != 'Other']['COSPAR_ID'].value_counts().sort_values(ascending=False)
print(f"\nTotal unique COSPAR IDs: {cospar_counts.shape[0]}")
print(f"Top COSPAR IDs:\n{cospar_counts.head(top_n)}")

top_cospars = cospar_counts.head(top_n).index.tolist()

# Label: top N stay as is, others → 'Other'
dfp['COSPAR_ID_COLORED'] = dfp['COSPAR_ID'].where(dfp['COSPAR_ID'].isin(top_cospars), other='Other')

print(f"\nCOSPAR_ID_COLORED counts:\n{dfp['COSPAR_ID_COLORED'].value_counts()}")

# Palette: top N get colors, 'Other' = grey
palette = sns.color_palette('tab20', n_colors=top_n)
palette_dict = dict(zip(top_cospars, palette))
palette_dict['Other'] = 'grey'

# --- PAIRPLOT ---
pp = sns.pairplot(
    dfp,
    hue='COSPAR_ID_COLORED',
    diag_kind='kde',
    plot_kws={'alpha': 0.6, 's': 8},
    corner=True,
    palette=palette_dict
)
pp.fig.suptitle(f"COSPAR Pairplot (Top {top_n}) {year_range}", y=1.02)
pp.savefig(os.path.join(plot_dir, f"pairplot_{year_range}_cospar_top{top_n}.png"))
plt.close(pp.fig)

# --- INDIVIDUAL PAIRS ---
for i, feat_x in enumerate(feature_names):
    for j, feat_y in enumerate(feature_names):
        if j >= i:
            continue

        plt.figure(figsize=(6, 6))
        ax = sns.scatterplot(
            data=dfp,
            x=feat_x,
            y=feat_y,
            hue='COSPAR_ID_COLORED',
            palette=palette_dict,
            s=8,
            alpha=0.7
        )
        plt.title(f"{feat_y} vs {feat_x} — {year_range} (Top {top_n})")
        plt.tight_layout()

        # Move legend outside
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles=handles, labels=labels,
                    bbox_to_anchor=(1.05, 1), loc='upper left',
                    borderaxespad=0.)

        out_name = f"scatter_{year_range}_{feat_y.replace(' ','')}_vs_{feat_x.replace(' ','')}_top{top_n}.png"
        plt.savefig(os.path.join(plot_dir, out_name), bbox_inches='tight')
        plt.close()


Processing Year Range: [2005, 2006, 2007, 2008]
dict_keys([287, 639, 93, 100, 97, 101, 98, 110, 461, 109, 349, 111, 631, 122, 555, 115, 127, 119, 125, 116, 291, 142, 139, 634, 556, 130, 442, 334, 426, 423, 265, 534, 124, 633, 491, 11, 632, 501, 366, 367, 485, 438, 144, 316, 2, 65, 373, 27, 370, 19, 21, 40, 636, 37, 36, 41, 237, 31, 429, 400, 244, 560, 34, 12, 14, 248, 264, 450, 17, 394, 33, 216, 18, 150, 16, 28, 635, 358, 446, 234, 243, 20, 346, 156, 255, 482, 256, 258, 257, 561, 323, 326, 420, 261, 398, 262, 361, 259, 266, 288, 271, 158, 162, 470, 337, 169, 173, 172, 436, 492, 197, 198, 207, 209, 365, 213, 369, 219, 182, 183, 225, 226, 227, 186, 437, 221, 205, 200, 201, 296, 281, 282, 285, 275, 278, 503, 500, 494, 44, 495, 498, 497, 523, 525, 524, 529, 530, 544, 551, 654, 706, 916, 802, 803, 917, 336, 559, 562, 558, 563])
All COSPAR-IDS form my geogto.dat file:
 1992-041C, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1992-072B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 19

data = [1992-041C, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1992-072B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1992-041C, 1991-075B, 1991-075B, 1985-056B, 1991-075B, 1988-040B, 1988-040B, 1985-056B, 1988-040B, 1988-040B, 1992-072B, 1988-040B, 1988-040B, 1988-040B, 1988-040B, 1985-056B, 1992-072B, 1992-072B, 1985-056B, 1988-040B, 1985-056B, 1985-056B, 1992-072B, 1988-040B, 1988-040B, 1991-075B, 1991-075B, 1991-075B, 1988-040B, 1988-040B, 1988-040B, 1985-056B, 1985-056B, 1988-040B, 1988-040B, 1988-040B, 1988-040B, 1985-056B, 1985-056B, 1985-056B, 1988-040B, 1988-040B, 1985-056B, 1988-040B, 1988-040B, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1991-803A, 1991-803A, 1991-803A, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1985-056B, 1985-056B, 1967-066G, 1967-066G, 1967-066G, 1991-803A, 1991-803A, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1988-040B, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1975-117A, 1967-066G, 1991-803A, 1967-066G, 1979-101A, 1988-040B, 1985-056B, 1979-101A, 1975-117A, 1975-117A, 1975-117A, 1991-803A, 1991-803A, 1988-040B, 1979-101A, 1979-101A, 1967-066G, 1967-066G, 1979-101A, 1979-101A, 1979-101A, 1988-040B, 1975-117A, 1975-117A, 1975-118C, 1975-118C, 1979-101A, 1975-118C, 1979-053C, 1975-123C, 1975-123C, 1975-123C, 1975-123C, 1975-123C, 1975-123C, 1975-123C, 1975-123C, 1975-123C, 1967-066G, 1980-018A, 1975-117A, 1975-123C, 1975-118C, 1975-118C, 1975-118C, 1975-118C, 1975-118C, 1975-123C, 1975-118C, 1975-118C, 1979-101A, 1988-040B, 1975-123C, 1975-123C, 1975-118C, 1979-087A, 1975-123C, 1975-123C, 1979-087A, 1975-117A, 1980-018A, 1975-117A, 1975-117A, 1979-087A, 1991-802A, 1975-123C, 1975-123C, 1977-092A, 1977-092A, 1979-087A, 1975-118C, 1979-087A, 1979-087A, 1979-087A, 1979-087A, 1975-117A, 1975-117A, 1979-087A, 1979-087A, 1979-087A, 1979-087A, 1979-087A, 1979-087A, 1979-087A, 1977-092A, 1975-118C, 1975-118C, 1977-092A, 1977-092A, 1982-093A, 1982-093A, 1975-123C, 1979-087A, 1975-117A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1975-118C, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1975-123C, 1982-093A, 1977-092A, 1977-092A, 1982-093A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1979-087A, 1975-123C, 1975-123C, 1975-123C, 1975-123C, 1982-093A, 1979-087A, 1982-093A, 1977-092A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1975-118C, 1977-092A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1979-087A, 1979-087A, 1979-087A, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1975-118C, 1975-118C, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1975-117A, 1975-117A, 1975-117A, 1979-101A, 1979-087A, 1975-118C, 1982-093A, 1982-093A, 1975-118C, 1975-118C, 1975-123C, 1975-123C, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1977-092A, 1975-123C, 1975-123C, 1991-803A, 1975-118C, 1975-118C, 1975-118C, 1982-093A, 1982-093A, 1975-123C, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1975-123C, 1975-123C, 1982-093A, 1982-093A, 1977-092A, 1977-092A, 1977-092A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1977-092A, 1977-092A, 1979-087A, 1979-087A, 1979-087A, 1979-087A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1975-117A, 1975-117A, 1975-117A, 1977-092A, 1982-093A, 1982-093A, 1991-067A, 1979-087A, 1979-087A, 1982-093A, 1979-053C, 1979-053C, 1979-053C, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1979-101A, 1979-101A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1979-087A, 1982-093A, 1982-093A, 1975-123C, 1975-123C, 1982-093A, 1975-123C, 1975-123C, 1979-053C, 1979-053C, 1979-053C, 1982-093A, 1979-053C, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-087A, 1979-087A, 1982-093A, 1982-093A, 1982-093A, 1979-053C, 1979-053C, 1982-093A, 1979-053C, 1979-087A, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1975-118C, 1979-053C, 1977-092A, 1977-092A, 1982-093A, 1982-093A, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1982-093A, 1975-117A, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-087A, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1975-117A, 1982-093A, 1979-087A, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1982-093A, 1982-093A, 1979-087A, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1982-093A, 1979-087A, 1979-101A, 1979-053C, 1979-053C, 1979-053C, 1975-118C, 1975-118C, 1975-118C, 1975-118C, 1979-053C, 1979-053C, 1979-053C, 1982-093A, 1982-093A, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1982-093A, 1979-087A, 1979-087A, 1979-087A, 1982-093A, 1975-117A, 1975-117A, 1975-117A, 1979-087A, 1975-123C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1979-101A, 1975-118C, 1975-118C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1979-101A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1977-092A, 1979-053C, 1979-101A, 1979-053C, 1979-053C, 1975-123C, 1982-093A, 1979-101A, 1979-101A, 1979-101A, 1979-101A, 1979-101A, 1979-101A, 1982-093A, 1982-093A, 1975-123C, 1975-123C, 1975-123C, 1975-123C, 1975-123C, 1975-123C, 1991-067A, 1991-067A, 1982-093A, 1975-123C, 1975-123C, 1982-093A, 1979-053C, 1979-053C, 1979-053C, 1975-117A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-101A, 1979-101A, 1979-053C, 1979-101A, 1975-123C, 1975-123C, 1975-123C, 1982-093A, 1982-093A, 1979-101A, 1982-093A, 1979-101A, 1979-101A, 1982-093A, 1975-117A, 1979-101A, 1975-118C, 1975-118C, 1979-101A, 1979-101A, 1979-101A, 1979-053C, 1979-053C, 1979-053C, 1979-101A, 1979-101A, 1975-117A, 1975-117A, 1979-101A, 1979-101A, 1982-093A, 1975-118C, 1975-118C, 1979-101A, 1979-101A, 1982-093A, 1979-101A, 1979-053C, 1979-087A, 1982-093A, 1975-117A, 1975-117A, 1979-101A, 1979-101A, 1979-101A, 1979-101A, 1982-093A, 1979-101A, 1979-101A, 1979-101A, 1982-093A, 1982-093A, 1982-093A, 1967-066G, 1967-066G, 1991-067A, 1991-067A, 1991-067A, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1975-117A, 1975-117A, 1975-117A, 1979-101A, 1979-053C, 1979-053C, 1979-101A, 1967-066G, 1967-066G, 1967-066G, 1980-018A, 1979-101A, 1975-123C, 1979-101A, 1979-101A, 1982-093A, 1982-093A, 1982-093A, 1979-101A, 1982-093A, 1982-093A, 1979-101A, 1975-117A, 1979-101A, 1979-101A, 1979-101A, 1982-093A, 1982-093A, 1982-093A, 1975-118C, 1975-118C, 1982-093A, 1975-117A, 1979-101A, 1982-093A, 1979-101A, 1975-118C, 1975-118C, 1975-117A, 1979-101A, 1991-067A, 1991-067A, 1975-117A, 1975-117A, 1975-123C, 1979-087A, 1979-087A, 1979-087A, 1975-117A, 1979-101A, 1982-093A, 1982-093A, 1982-093A, 1975-123C, 1979-053C, 1975-117A, 1979-101A, 1975-117A, 1975-117A, 1979-101A, 1979-101A, 1975-117A, 1979-101A, 1979-101A, 1982-093A, 1982-093A, 1975-118C, 1975-118C, 1982-093A, 1982-093A, 1979-101A, 1982-093A, 1982-093A, 1977-092A, 1977-092A, 1967-066G, 1967-066G, 1975-123C, 1975-123C, 1975-118C, 1975-117A, 1982-093A, 1979-053C, 2002-051B, 1988-040B, 1991-075B, 1991-075B, 1992-041C, 1992-041C, 1992-072B, 1992-041C, 1991-075B, 1991-075B, 1992-072B, 1991-075B, 1985-056B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1992-072B, 1991-075B, 1991-075B, 1992-072B, 1991-075B, 1992-072B, 1988-040B, 1991-075B, 1992-072B, 1992-072B, 1992-072B, 1988-040B, 1988-040B, 1985-056B, 1985-056B, 1992-072B, 1988-040B, 1992-072B, 1992-041C, 1992-041C, 1992-072B, 1985-056B, 1985-056B, 1992-072B, 1992-072B, 1988-040B, 1988-040B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1988-040B, 1985-056B, 1992-072B, 1992-072B, 1988-040B, 1985-056B, 1992-072B, 1991-075B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1988-040B, 1988-040B, 1985-056B, 1985-056B, 1985-056B, 1988-040B, 1985-056B, 1985-056B, 1979-101A, 1985-056B, 1988-040B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1966-053J, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1985-056B, 1985-056B, 1985-056B, 1967-066G, 1988-040B, 1988-040B, 1991-803A, 1988-040B, 1985-056B, 1979-101A, 1988-040B, 1988-040B, 1988-040B, 1991-803A, 1991-803A, 1988-040B, 1988-040B, 1985-056B, 1979-101A, 1985-056B, 1975-117A, 1975-117A, 1991-803A, 1988-040B, 1988-040B, 1988-040B, 1979-101A, 1979-101A, 1979-101A, 1985-056B, 1975-117A, 1979-101A, 1979-101A, 1975-118C, 1975-118C, 1975-118C, 1975-118C, 1975-123C, 1975-118C, 1975-118C, 1985-056B, 1975-123C, 1979-101A, 1975-123C, 1975-117A, 1979-087A, 1977-092A, 1979-087A, 1982-093A, 1975-117A, 1982-093A, 1982-093A, 1975-117A, 1982-093A, 1982-093A, 1975-123C, 1982-093A, 1982-093A, 1975-117A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1975-118C, 1977-092A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1980-018A, 1982-093A, 1982-093A, 1982-093A, 1979-101A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1977-092A, 1982-093A, 1979-087A, 1982-093A, 1982-093A, 1975-118C, 1975-117A, 1975-117A, 1982-093A, 1982-093A, 1979-087A, 1982-093A, 1982-093A, 1975-117A, 1975-117A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1980-018A, 1982-093A, 1982-093A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1979-101A, 1982-093A, 1975-123C, 1973-040B, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1979-053C, 1975-117A, 1979-101A, 1982-093A, 1982-093A, 1979-053C, 1982-093A, 1982-093A, 1979-101A, 1979-053C, 1979-053C, 1982-093A, 1982-093A, 1975-117A, 1975-117A, 1975-117A, 1982-093A, 1979-053C, 1975-123C, 1982-093A, 1982-093A, 1975-117A, 1975-118C, 1979-053C, 1982-093A, 1982-093A, 1982-093A, 1975-117A, 1982-093A, 1982-093A, 1979-053C, 1979-053C, 1979-087A, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1982-093A, 1979-053C, 1979-053C, 1982-093A, 1982-093A, 1979-053C, 1985-056B, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1988-040B, 1975-118C, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1979-053C, 1979-053C, 1982-093A, 1975-123C, 1975-123C, 1982-093A, 1979-053C, 1979-087A, 1979-053C, 1979-087A, 1975-118C, 1975-118C, 1975-118C, 1980-018A, 1977-092A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1979-101A, 1979-101A, 1979-053C, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1979-053C, 1982-093A, 1979-087A, 1979-053C, 1979-053C, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1980-018A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1979-053C, 1982-093A, 1982-093A, 1979-101A, 1982-093A, 1979-053C, 1979-053C, 1979-053C, 1979-101A, 1979-101A, 1979-101A, 1985-056B, 1975-117A, 1979-053C, 1982-093A, 1982-093A, 1979-087A, 1979-087A, 1982-093A, 1982-093A, 1979-101A, 1975-118C, 1975-118C, 1982-093A, 1979-101A, 1979-101A, 1982-093A, 1982-093A, 1979-101A, 1979-101A, 1975-117A, 1982-093A, 1979-053C, 1979-053C, 1967-066G, 1979-101A, 1980-018A, 1975-118C, 1979-101A, 1979-101A, 1982-093A, 1982-093A, 1979-053C, 1975-123C, 1979-101A, 1982-093A, 1979-053C, 1979-053C, 1982-093A, 1979-101A, 1979-101A, 1979-101A, 1979-101A, 1979-101A, 1982-093A, 1979-101A, 1979-101A, 1979-087A, 1979-101A, 1979-053C, 1967-066G, 1982-093A, 1982-093A, 1982-093A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1979-101A, 1982-093A, 1979-101A, 1979-101A, 1975-117A, 1982-093A, 1979-101A, 1975-117A, 1975-117A, 1975-117A, 1975-118C, 1979-053C, 1979-053C, 1979-101A, 1979-053C, 1991-067A, 1979-053C, 1979-101A, 1975-118C, 1975-118C, 1979-101A, 1975-118C, 1979-101A, 1975-123C, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1985-056B, 1975-117A, 1975-117A, 1982-093A, 1982-093A, 1975-123C, 1979-053C, 1979-053C, 1982-093A, 1985-056B, 1982-093A, 1979-053C, 1979-053C, 1979-101A, 1979-101A, 1975-123C, 1975-117A, 1979-087A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1979-101A, 1979-101A, 1975-117A, 1979-101A, 1982-093A, 1975-118C, 1975-117A, 1982-093A, 1975-117A, 1992-041C, 1988-040B, 1988-040B, 1992-041C, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1985-056B, 1991-075B, 1992-072B, 1991-075B, 1991-075B, 1992-041C, 1991-075B, 1992-072B, 1992-072B, 1985-056B, 1988-040B, 1991-075B, 1988-040B, 1992-072B, 1988-040B, 1992-072B, 1985-056B, 1985-056B, 1992-072B, 1992-072B, 1992-072B, 1992-072B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1992-072B, 1988-040B, 1988-040B, 1992-072B, 1985-056B, 1988-040B, 1985-056B, 1967-066G, 1967-066G, 1967-066G, 1966-053J, 1967-066G, 1967-066G, 1967-066G, 1966-053J, 1967-066G, 1991-803A, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1985-056B, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1985-056B, 1979-101A, 1967-066G, 1988-040B, 1988-040B, 1988-040B, 1967-066G, 1988-040B, 1979-101A, 1979-101A, 1979-101A, 1979-101A, 1988-040B, 1988-040B, 1967-066G, 1975-117A, 1979-101A, 1988-040B, 1985-056B, 1975-117A, 1979-101A, 1979-101A, 1975-117A, 1975-118C, 1975-123C, 1975-123C, 1975-123C, 1975-123C, 1967-066G, 1967-066G, 1988-040B, 1975-117A, 1975-117A, 1975-123C, 1975-118C, 1975-118C, 1975-117A, 1979-087A, 1975-117A, 1977-092A, 1975-117A, 1975-117A, 1979-087A, 1979-087A, 1979-087A, 1979-087A, 1977-092A, 1977-092A, 1977-092A, 1982-093A, 1979-087A, 1982-093A, 1977-092A, 1979-087A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1975-123C, 1982-093A, 1982-093A, 1979-101A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1975-123C, 1982-093A, 1982-093A, 1967-066G, 1975-118C, 1982-093A, 1977-092A, 1982-093A, 1982-093A, 1977-092A, 1979-087A, 1982-093A, 1982-093A, 1982-093A, 1979-101A, 1975-117A, 1979-087A, 1979-101A, 1980-018A, 1982-093A, 1982-093A, 1975-118C, 1975-117A, 1982-093A, 1982-093A, 1979-101A, 1980-018A, 1982-093A, 1982-093A, 1975-117A, 1979-087A, 1977-092A, 1979-087A, 1982-093A, 1975-117A, 1975-118C, 1975-117A, 1979-087A, 1975-117A, 1975-117A, 1975-117A, 1975-117A, 1982-093A, 1975-123C, 1982-093A, 1975-117A, 1982-093A, 1979-087A, 1975-117A, 1979-101A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1977-092A, 1975-123C, 1979-053C, 1988-040B, 1988-040B, 1979-101A, 1967-066G, 1979-053C, 1975-123C, 1982-093A, 1982-093A, 1975-117A, 1975-117A, 1979-053C, 1979-087A, 1979-053C, 1979-053C, 1982-093A, 1982-093A, 1975-118C, 1979-087A, 1979-053C, 1977-092A, 1988-040B, 1982-093A, 1982-093A, 1979-053C, 1982-093A, 1982-093A, 1979-053C, 1982-093A, 1979-101A, 1979-053C, 1982-093A, 1977-092A, 1979-087A, 1982-093A, 1982-093A, 1979-101A, 1982-093A, 1979-053C, 1982-093A, 1982-093A, 1979-101A, 1975-118C, 1975-118C, 1975-117A, 1975-118C, 1979-053C, 1979-101A, 1979-101A, 1975-123C, 1982-093A, 1979-101A, 1982-093A, 1979-087A, 1979-087A, 1979-101A, 1980-018A, 1982-093A, 1982-093A, 1982-093A, 1979-053C, 1979-101A, 1975-117A, 1979-101A, 1979-101A, 1982-093A, 1979-101A, 1979-101A, 1979-053C, 1979-101A, 1979-053C, 1979-101A, 1979-101A, 1979-101A, 1975-123C, 1979-087A, 1979-101A, 1979-101A, 1979-101A, 1979-101A, 1979-101A, 1979-101A, 1979-101A, 1979-101A, 1979-101A, 1975-117A, 1979-101A, 1979-053C, 1979-101A, 1975-117A, 1979-101A, 1979-101A, 1975-118C, 1979-101A, 1979-101A, 1979-101A, 1979-101A, 1979-053C, 1991-067A, 1979-101A, 1979-087A, 1967-066G, 1979-053C, 1979-101A, 1979-087A, 1979-087A, 1975-117A, 1975-117A, 1979-101A, 1982-093A, 1982-093A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1979-053C, 1979-053C, 1979-101A, 1979-101A, 1979-101A, 1979-053C, 1979-101A, 1979-087A, 1979-101A, 1975-123C, 1975-117A, 1979-101A, 1979-087A, 1979-101A, 1979-101A, 1979-101A, 1979-101A, 1975-118C, 1975-117A, 1979-101A, 1979-101A, 1975-123C, 1979-101A, 1985-056B, 1982-093A, 1975-123C, 1977-092A, 1977-092A, 1991-067A, 1979-101A, 1979-101A, 1979-053C, 1975-118C, 1979-053C, 1982-093A, 1979-053C, 1988-040B, 1988-040B, 1988-040B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1988-040B, 1992-072B, 1992-072B, 1992-041C, 1985-056B, 1991-075B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1991-075B, 1991-075B, 1985-056B, 1992-072B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1992-072B, 1985-056B, 1985-056B, 1967-066G, 1967-066G, 1985-056B, 1985-056B, 1967-066G, 1991-803A, 1967-066G, 1967-066G, 1967-066G, 1979-101A, 1975-117A, 1988-040B, 1975-117A, 1975-117A, 1988-040B, 1975-117A, 1975-118C, 1975-117A, 1975-117A, 1975-117A, 1975-118C, 1975-118C, 1975-118C, 1975-118C, 1975-118C, 1975-118C, 1975-118C, 1975-118C, 1975-118C, 1975-118C, 1975-118C, 1975-118C, 1975-118C, 1975-123C, 1975-118C, 1975-118C, 1982-093A, 1975-117A, 1975-117A, 1975-117A, 1975-117A, 1975-117A, 1982-093A, 1982-093A, 1979-053C, 1980-018A, 1980-018A, 1980-018A, 1975-117A, 1982-093A, 1977-092A, 1977-092A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1975-118C, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1975-118C, 1977-092A, 1977-092A, 1982-093A, 1975-118C, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1979-087A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1975-117A, 1982-093A, 1979-087A, 1975-123C, 1975-123C, 1975-123C, 1979-087A, 1980-018A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1975-117A, 1975-117A, 1975-117A, 1979-087A, 1979-087A, 1979-087A, 1991-067A, 1982-093A, 1975-123C, 1975-123C, 1975-123C, 1975-117A, 1979-087A, 1979-087A, 1975-118C, 1975-118C, 1975-118C, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1977-092A, 1975-118C, 1979-087A, 1975-123C, 1975-123C, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1979-053C, 1979-087A, 1979-087A, 1979-087A, 1977-092A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1975-123C, 1979-053C, 1982-093A, 1982-093A, 1977-092A, 1979-087A, 1979-087A, 1979-087A, 1979-053C, 1977-092A, 1979-053C, 1979-053C, 1982-093A, 1982-093A, 1977-092A, 1979-087A, 1982-093A, 1982-093A, 1979-087A, 1979-087A, 1975-117A, 1979-053C, 1979-053C, 1982-093A, 1982-093A, 1975-117A, 1991-802A, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1985-105A, 1985-105A, 1979-087A, 1975-117A, 1975-117A, 1979-053C, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1977-092A, 1979-087A, 1979-087A, 1982-093A, 1977-092A, 1982-093A, 1982-093A, 1977-092A, 1982-093A, 1979-053C, 1975-117A, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1975-123C, 1979-087A, 1975-123C, 1975-123C, 1975-123C, 1975-123C, 1982-093A, 1979-053C, 1979-053C, 1975-117A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1985-105A, 1982-093A, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1975-118C, 1975-118C, 1975-118C, 1982-093A, 1982-093A, 1982-093A, 1979-053C, 1979-053C, 1982-093A, 1982-093A, 1979-053C, 1979-053C, 1979-053C, 1975-123C, 1979-053C, 1975-123C, 1979-087A, 1979-087A, 1979-087A, 1979-053C, 1979-053C, 1975-123C, 1975-123C, 1975-123C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1975-117A, 1975-117A, 1977-092A, 1977-092A, 1977-092A, 1977-092A, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1975-118C, 1979-101A, 1982-093A, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1982-093A, 1982-093A, 1975-118C, 1979-053C, 1982-093A, 1982-093A, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1977-092A, 1979-053C, 1975-123C, 1982-093A, 1982-093A, 1975-123C, 1975-123C, 1979-101A, 1977-092A, 1975-123C, 1975-123C, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1979-101A, 1975-117A, 1975-123C, 1975-123C, 1975-123C, 1979-053C, 1975-117A, 1975-117A, 1975-117A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1985-105A, 1985-105A, 1975-118C, 1975-118C, 1975-118C, 1975-118C, 1977-092A, 1975-123C, 1975-123C, 1979-053C, 1975-123C, 1991-802A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1973-040B, 1975-123C, 1975-123C, 1975-123C, 1982-093A, 1975-123C, 1975-123C, 1975-123C, 1975-123C, 1982-093A, 1975-118C, 1975-118C, 1975-118C, 1979-053C, 1975-117A, 1979-053C, 1979-053C, 1975-123C, 1979-053C, 1975-118C, 1975-118C, 1975-123C, 1982-093A, 1975-123C, 1975-123C, 1977-092A, 1977-092A, 1979-053C, 1982-093A, 1975-118C, 1979-053C, 1979-053C, 1979-053C, 1982-093A, 1982-093A, 1975-117A, 1991-067A, 1991-067A, 1980-018A, 1980-018A, 1980-018A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1975-118C, 1975-118C, 1980-018A, 1967-066G, 1991-067A, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1979-053C, 1975-117A, 1975-123C, 1975-123C, 1975-123C, 1975-118C, 1975-123C, 1975-123C, 1975-118C, 1979-087A, 1975-118C, 1975-117A, 1982-093A, 1982-093A, 1975-117A, 1982-093A, 1982-093A, 1975-117A, 1979-087A, 1975-117A, 1979-053C, 1979-053C, 1979-053C, 1975-117A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1975-117A, 1979-087A, 1975-117A, 1975-117A, 1975-117A, 1975-117A, 1979-053C, 1980-018A, 1979-053C, 1975-117A, 1979-053C, 1982-093A, 1975-117A, 1975-117A, 1973-040B, 1975-123C, 1975-123C, 1975-117A, 1975-118C, 1975-118C, 1985-105A, 1975-123C, 1982-093A, 1975-123C, 1979-053C, 1975-117A, 1977-092A, 1975-117A, 1975-117A, 1975-123C, 1982-093A, 1985-056B, 1975-117A, 1975-117A, 1985-056B, 1985-056B, 1975-118C, 1982-093A, 1967-066G, 1967-066G, 1967-066G, 1975-117A, 1979-101A, 1979-087A, 1979-087A, 1979-053C, 1991-067A, 1991-067A, 1967-066G, 1975-118C, 1975-117A, 1982-093A, 1991-075B, 1992-041C, 1992-041C, 1992-041C, 1991-075B, 1991-075B, 1991-075B, 1992-041C, 1988-040B, 1988-040B, 1991-075B, 1991-075B, 1988-040B, 1988-040B, 1992-072B, 1992-041C, 1985-056B, 1988-040B, 1992-072B, 1992-072B, 1988-040B, 1985-056B, 1985-056B, 1992-072B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1992-072B, 1992-072B, 1985-056B, 1985-056B, 1985-056B, 1992-072B, 1992-072B, 1992-072B, 1992-072B, 1992-072B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1967-066G, 1988-040B, 1988-040B, 1988-040B, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1985-056B, 1985-056B, 1985-056B, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1979-101A, 1991-803A, 1988-040B, 1988-040B, 1988-040B, 1975-117A, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1991-803A, 1967-066G, 1967-066G, 1979-101A, 1988-040B, 1967-066G, 1975-117A, 1979-101A, 1967-066G, 1979-101A, 1967-066G, 1967-066G, 1988-040B, 1991-803A, 1991-803A, 1991-803A, 1979-101A, 1988-040B, 1988-040B, 1967-066G, 1988-040B, 1988-040B, 1988-040B, 1988-040B, 1975-117A, 1979-101A, 1985-056B, 1967-066G, 1966-053J, 1966-053J, 1967-066G, 1985-056B, 1985-056B, 1988-040B, 1988-040B, 1988-040B, 1979-101A, 1979-101A, 1985-056B, 1985-056B, 1985-056B, 1979-101A, 1975-117A, 1967-066G, 1975-117A, 1975-117A, 1979-101A, 1975-118C, 1979-101A, 1988-040B, 1975-118C, 1988-040B, 1975-118C, 1975-123C, 1979-101A, 1975-123C, 1979-101A, 1979-101A, 1988-040B, 1975-118C, 1975-117A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1975-123C, 1988-040B, 1982-093A, 1982-093A, 1979-087A, 1975-117A, 1975-117A, 1975-118C, 1982-093A, 1982-093A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1975-123C, 1975-118C, 1979-101A, 1982-093A, 1982-093A, 1982-093A, 1975-118C, 1975-118C, 1975-123C, 1977-092A, 1979-101A, 1979-101A, 1979-101A, 1977-092A, 1975-117A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1977-092A, 1982-093A, 1975-123C, 1977-092A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1979-087A, 1979-087A, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1975-123C, 1979-087A, 1979-087A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1979-101A, 1979-087A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1979-101A, 1977-092A, 1975-117A, 1975-117A, 1980-018A, 1980-018A, 1980-018A, 1982-093A, 1975-123C, 1979-087A, 1975-123C, 1975-118C, 1975-118C, 1982-093A, 1982-093A, 1979-087A, 1982-093A, 1979-087A, 1982-093A, 1975-118C, 1975-118C, 1979-053C, 1982-093A, 1979-087A, 1979-087A, 1982-093A, 1975-117A, 1975-123C, 1975-118C, 1975-117A, 1975-118C, 1982-093A, 1979-101A, 1975-123C, 1975-123C, 1980-018A, 1979-053C, 1975-123C, 1979-053C, 1980-018A, 1980-018A, 1980-018A, 1982-093A, 1977-092A, 1982-093A, 1975-118C, 1979-087A, 1979-087A, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1975-117A, 1975-123C, 1975-123C, 1979-053C, 1977-092A, 1975-117A, 1979-053C, 1975-123C, 1975-123C, 1979-101A, 1985-105A, 1985-105A, 1977-092A, 1979-087A, 1979-087A, 1982-093A, 1982-093A, 1979-087A, 1982-093A, 1982-093A, 1982-093A, 1979-101A, 1975-123C, 1975-123C, 1977-092A, 1979-087A, 1979-087A, 1979-087A, 1982-093A, 1982-093A, 1979-053C, 1975-117A, 1979-053C, 1980-018A, 1980-018A, 1979-053C, 1982-093A, 1979-053C, 1975-117A, 1979-087A, 1982-093A, 1979-087A, 1982-093A, 1982-093A, 1979-101A, 1985-105A, 1975-117A, 1985-105A, 1982-093A, 1982-093A, 1979-087A, 1979-053C, 1979-053C, 1975-123C, 1975-118C, 1979-053C, 1979-053C, 1979-101A, 1979-053C, 1982-093A, 1975-123C, 1979-053C, 1979-053C, 1977-092A, 1975-118C, 1975-118C, 1988-040B, 1982-093A, 1975-117A, 1975-117A, 1982-093A, 1979-087A, 1977-092A, 1975-117A, 1975-117A, 1975-123C, 1975-117A, 1982-093A, 1979-087A, 1979-087A, 1982-093A, 1979-053C, 1975-118C, 1982-093A, 1982-093A, 1979-053C, 1975-118C, 1977-092A, 1975-117A, 1982-093A, 1982-093A, 1975-123C, 1982-093A, 1982-093A, 1977-092A, 1979-087A, 1977-092A, 1982-093A, 1975-118C, 1979-053C, 1977-092A, 1975-123C, 1982-093A, 1975-123C, 1975-123C, 1982-093A, 1979-101A, 1980-018A, 1977-092A, 1977-092A, 1977-092A, 1982-093A, 1975-117A, 1975-117A, 1982-093A, 1979-101A, 1975-123C, 1979-101A, 1979-087A, 1979-087A, 1975-123C, 1975-123C, 1979-087A, 1979-101A, 1979-087A, 1982-093A, 1979-087A, 1979-101A, 1975-123C, 1975-117A, 1975-123C, 1975-123C, 1975-117A, 1979-087A, 1975-117A, 1979-087A, 1979-053C, 1979-053C, 1977-092A, 1982-093A, 1982-093A, 1975-123C, 1982-093A, 1982-093A, 1982-093A, 1979-053C, 1979-087A, 1975-118C, 1977-092A, 1977-092A, 1979-087A, 1979-053C, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1975-117A, 1982-093A, 1979-087A, 1985-105A, 1985-105A, 1985-105A, 1982-093A, 1991-067A, 1967-066G, 1967-066G, 1991-067A, 1979-053C, 1979-053C, 1977-092A, 1979-087A, 1982-093A, 1979-101A, 1975-118C, 1979-087A, 1975-117A, 1975-117A, 1967-066G, 1982-093A, 1982-093A, 1975-117A, 1975-118C, 1975-123C, 1975-118C, 1977-092A, 1982-093A, 1979-053C, 1979-087A, 1975-117A, 1975-123C, 1975-118C, 1979-087A, 1982-093A, 1979-087A, 1975-117A, 1979-087A, 1975-117A, 1979-087A, 1979-053C, 1979-087A, 1967-066G, 1967-066G, 1982-093A, 1982-093A, 1982-093A, 1979-053C, 1975-123C, 1975-118C, 1975-117A, 1985-105A, 1985-105A, 1985-105A, 1985-105A, 1979-101A, 1975-118C, 1975-118C, 1985-105A, 1979-101A, 1982-093A, 1979-101A, 1979-087A, 1975-117A, 1975-117A, 1982-093A, 1985-056B, 1979-101A, 1975-123C, 1982-093A, 1982-093A, 1979-087A, 1985-105A, 1985-105A, 1982-093A, 1979-101A, 1985-056B, 1982-093A, 1982-093A, 1982-093A, 1979-101A, 1982-093A, 1982-093A, 1967-066G, 1967-066G, 1975-118C, 1979-101A, 1979-101A, 1979-101A, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1975-117A, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1992-041C, 1991-075B, 1991-075B, 1992-041C, 1988-040B, 1992-072B, 1992-072B, 1988-040B, 1988-040B, 1991-075B, 1988-040B, 1988-040B, 1992-072B, 1991-075B, 1992-072B, 1992-041C, 1992-072B, 1992-041C, 1985-056B, 1985-056B, 1992-072B, 1992-072B, 1985-056B, 1992-072B, 1985-056B, 1992-072B, 1985-056B, 1988-040B, 1992-072B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1988-040B, 1988-040B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1966-053J, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1991-803A, 1967-066G, 1967-066G, 1967-066G, 1966-053J, 1966-053J, 1967-066G, 1967-066G, 1967-066G, 1988-040B, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1985-056B, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1985-056B, 1967-066G, 1988-040B, 1967-066G, 1988-040B, 1988-040B, 1979-101A, 1979-101A, 1985-056B, 1988-040B, 1988-040B, 1975-117A, 1966-053J, 1985-056B, 1979-101A, 1967-066G, 1966-053J, 1985-056B, 1967-066G, 1979-101A, 1975-118C, 1979-101A, 1967-066G, 1980-018A, 1975-118C, 1975-118C, 1975-118C, 1988-040B, 1975-118C, 1975-123C, 1975-118C, 1975-123C, 1975-123C, 1982-093A, 1975-118C, 1975-117A, 1982-093A, 1975-117A, 1975-117A, 1975-118C, 1980-018A, 1982-093A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1985-056B, 1982-093A, 1967-066G, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1982-093A, 1982-093A, 1982-093A, 1975-118C, 1982-093A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1977-092A, 1982-093A, 1979-087A, 1982-093A, 1975-117A, 1982-093A, 1982-093A, 1975-123C, 1975-117A, 1982-093A, 1982-093A, 1977-092A, 1967-066G, 1975-117A, 1982-093A, 1982-093A, 1979-087A, 1991-067A, 1982-093A, 1982-093A, 1975-117A, 1979-087A, 1975-118C, 1975-118C, 1975-118C, 1982-093A, 1979-087A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1975-118C, 1975-117A, 1979-087A, 1982-093A, 1975-117A, 1977-092A, 1979-101A, 1982-093A, 1982-093A, 1979-087A, 1979-053C, 1979-101A, 1977-092A, 1980-018A, 1982-093A, 1979-087A, 1975-117A, 1979-087A, 1975-117A, 1979-053C, 1975-123C, 1982-093A, 1979-087A, 1991-802A, 1975-123C, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1979-087A, 1979-087A, 1982-093A, 1975-118C, 1975-118C, 1982-093A, 1979-053C, 1979-053C, 1979-101A, 1982-093A, 1982-093A, 1979-087A, 1982-093A, 1982-093A, 1982-093A, 1979-101A, 1979-053C, 1975-117A, 1982-093A, 1979-053C, 1975-123C, 1975-123C, 1982-093A, 1979-087A, 1979-087A, 1975-117A, 1975-117A, 1979-087A, 1985-105A, 1979-087A, 1979-101A, 1979-087A, 1982-093A, 1977-092A, 1979-053C, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1985-105A, 1977-092A, 1975-117A, 1977-092A, 1979-053C, 1977-092A, 1982-093A, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-101A, 1979-101A, 1979-101A, 1982-093A, 1975-123C, 1982-093A, 1982-093A, 1975-117A, 1982-093A, 1975-123C, 1979-053C, 1975-117A, 1979-053C, 1980-018A, 1979-101A, 1991-067A, 1979-101A, 1979-087A, 1979-101A, 1979-087A, 1979-101A, 1982-093A, 1979-087A, 1975-118C, 1975-117A, 1979-087A, 1975-117A, 1975-117A, 1977-092A, 1975-123C, 1977-092A, 1979-087A, 1979-087A, 1979-087A, 1979-087A, 1979-053C, 1975-117A, 1975-117A, 1975-117A, 1982-093A, 1979-101A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1975-118C, 1975-118C, 1991-067A, 1991-067A, 1967-066G, 1967-066G, 1975-117A, 1975-123C, 1975-123C, 1975-117A, 1975-117A, 1979-101A, 1975-118C, 1979-087A, 1979-087A, 1979-101A, 1967-066G, 1975-117A, 1982-093A, 1975-118C, 1979-087A, 1979-087A, 1979-101A, 1980-018A, 1980-018A, 1975-123C, 1975-123C, 1985-105A, 1982-093A, 1985-105A, 1982-093A, 1975-123C, 1979-087A, 1979-101A, 1975-117A, 1975-123C, 1979-101A, 1982-093A, 1985-105A, 1975-117A, 1980-018A, 1982-093A, 1977-092A, 1975-117A, 1967-066G, 1967-066G, 1982-093A, 1979-101A, 1977-092A, 1979-087A, 1979-101A, 1979-087A, 1979-101A, 1967-066G, 1967-066G, 1975-117A, 1991-075B, 1991-075B, 1992-072B, 1988-040B, 1992-041C, 1985-056B, 1985-056B, 1992-072B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1988-040B, 1985-056B, 1985-056B, 1985-056B, 1992-072B, 1992-072B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1975-117A, 1967-066G, 1975-117A, 1967-066G, 1967-066G, 1979-101A, 1975-117A, 1975-117A, 1988-040B, 1988-040B, 1975-117A, 1975-117A, 1979-053C, 1975-117A, 1975-117A, 1975-117A, 1975-117A, 1982-093A, 1982-093A, 1975-117A, 1982-093A, 1982-093A, 1975-118C, 1982-093A, 1975-118C, 1975-118C, 1975-118C, 1975-123C, 1975-118C, 1975-117A, 1975-117A, 1980-018A, 1980-018A, 1982-093A, 1975-117A, 1982-093A, 1977-092A, 1975-118C, 1977-092A, 1977-092A, 1977-092A, 1982-093A, 1975-118C, 1982-093A, 1975-118C, 1975-118C, 1975-118C, 1982-093A, 1982-093A, 1977-092A, 1977-092A, 1982-093A, 1977-092A, 1977-092A, 1979-087A, 1975-117A, 1975-117A, 1982-093A, 1979-087A, 1979-087A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1991-803A, 1975-123C, 1979-087A, 1979-087A, 1979-087A, 1979-087A, 1982-093A, 1979-087A, 1979-087A, 1979-087A, 1979-087A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1979-087A, 1982-093A, 1975-117A, 1975-118C, 1979-087A, 1979-087A, 1975-118C, 1975-118C, 1982-093A, 1975-117A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1979-053C, 1979-087A, 1979-087A, 1979-087A, 1977-092A, 1977-092A, 1982-093A, 1979-087A, 1975-117A, 1975-117A, 1975-117A, 1975-117A, 1975-117A, 1967-066G, 1977-092A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1979-087A, 1975-123C, 1975-123C, 1982-093A, 1982-093A, 1977-092A, 1979-087A, 1979-087A, 1975-118C, 1975-117A, 1975-117A, 1975-117A, 1975-117A, 1977-092A, 1982-093A, 1982-093A, 1977-092A, 1982-093A, 1975-117A, 1975-117A, 1975-117A, 1975-117A, 1975-117A, 1977-092A, 1975-117A, 1979-053C, 1982-093A, 1982-093A, 1979-087A, 1977-092A, 1982-093A, 1979-087A, 1979-087A, 1979-087A, 1982-093A, 1975-118C, 1991-067A, 1991-067A, 1975-117A, 1975-117A, 1982-093A, 1982-093A, 1975-118C, 1975-118C, 1975-118C, 1975-118C, 1975-118C, 1975-118C, 1982-093A, 1975-117A, 1975-117A, 1975-117A, 1975-117A, 1979-087A, 1982-093A, 1977-092A, 1977-092A, 1977-092A, 1985-105A, 1975-118C, 1975-118C, 1975-117A, 1975-117A, 1975-117A, 1982-093A, 1975-123C, 1975-123C, 1991-067A, 1991-067A, 1980-018A, 1982-093A, 1979-087A, 1979-087A, 1982-093A, 1982-093A, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1975-117A, 1975-123C, 1979-053C, 1975-117A, 1979-053C, 1979-087A, 1979-053C, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1975-117A, 1975-117A, 1975-117A, 1975-117A, 1982-093A, 1975-117A, 1979-087A, 1982-093A, 1975-117A, 1979-053C, 1979-053C, 1975-123C, 1979-053C, 1975-118C, 1979-087A, 1982-093A, 1982-093A, 1979-053C, 1982-093A, 1982-093A, 1975-117A, 1975-117A, 1975-123C, 1975-123C, 1975-123C, 1975-123C, 1980-018A, 1982-093A, 1979-087A, 1975-117A, 1979-053C, 1979-053C, 1979-053C, 1982-093A, 1979-053C, 1979-053C, 1982-093A, 1975-123C, 1975-123C, 1982-093A, 1975-117A, 1975-117A, 1975-123C, 1975-117A, 1979-053C, 1979-053C, 1982-093A, 1979-087A, 1979-053C, 1979-087A, 1975-118C, 1975-117A, 1975-117A, 1975-117A, 1975-117A, 1975-117A, 1979-053C, 1979-087A, 1977-092A, 1975-117A, 1975-117A, 1975-117A, 1982-093A, 1975-123C, 1977-092A, 1975-118C, 1975-118C, 1975-118C, 1975-118C, 1975-118C, 1979-053C, 1979-053C, 1977-092A, 1979-087A, 1979-053C, 1979-053C, 1975-118C, 1975-118C, 1982-093A, 1975-123C, 1982-093A, 1975-117A, 1977-092A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1975-123C, 1975-123C, 1977-092A, 1975-118C, 1979-053C, 1977-092A, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1975-117A, 1979-087A, 1979-087A, 1982-093A, 1975-117A, 1975-117A, 1975-117A, 1975-117A, 1975-123C, 1975-123C, 1982-093A, 1977-092A, 1982-093A, 1982-093A, 1980-018A, 1980-018A, 1979-053C, 1979-053C, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1982-093A, 1982-093A, 1991-067A, 1991-067A, 1975-117A, 1975-117A, 1982-093A, 1991-067A, 1975-123C, 1975-123C, 1975-118C, 1975-117A, 1975-117A, 1975-117A, 1975-117A, 1979-053C, 1975-123C, 1982-093A, 1982-093A, 1975-118C, 1975-118C, 1982-093A, 1975-117A, 1975-117A, 1975-117A, 1975-118C, 1979-087A, 1985-105A, 1975-117A, 1982-093A, 1982-093A, 1975-117A, 1980-018A, 1975-123C, 1982-093A, 1975-118C, 1975-117A, 1982-093A, 1982-093A, 1979-087A, 1975-118C, 1982-093A, 1967-066G, 1975-123C, 1975-123C, 1975-123C, 1975-117A, 1975-117A, 1977-092A, 1975-118C, 1975-118C, 1975-117A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1975-117A, 1975-117A, 1977-092A, 1982-093A, 1975-117A, 1975-117A, 1979-101A, 1967-066G, 1979-053C, 1985-105A, 1982-093A, 1982-093A, 1979-053C, 1991-075B, 1988-040B, 1992-041C, 1992-072B, 1988-040B, 1992-072B, 1991-075B, 1992-041C, 1988-040B, 1992-041C, 1988-040B, 1988-040B, 1992-072B, 1985-056B, 1992-072B, 1992-072B, 1992-072B, 1985-056B, 1985-056B, 1992-072B, 1988-040B, 1988-040B, 1988-040B, 1985-056B, 1985-056B, 1992-072B, 1985-056B, 1992-072B, 1985-056B, 1985-056B, 1992-072B, 1992-072B, 1992-072B, 1988-040B, 1988-040B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1992-072B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1985-056B, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1979-101A, 1967-066G, 1967-066G, 1967-066G, 1991-803A, 1991-803A, 1967-066G, 1967-066G, 1979-101A, 1967-066G, 1967-066G, 1975-117A, 1975-117A, 1967-066G, 1967-066G, 1979-101A, 1988-040B, 1985-056B, 1985-056B, 1985-056B, 1967-066G, 1967-066G, 1985-056B, 1985-056B, 1966-053J, 1985-056B, 1985-056B, 1991-803A, 1991-803A, 1979-101A, 1975-117A, 1975-117A, 1991-803A, 1967-066G, 1966-053J, 1988-040B, 1967-066G, 1967-066G, 1975-117A, 1979-101A, 1975-117A, 1985-056B, 1985-056B, 1967-066G, 1975-118C, 1980-018A, 1975-117A, 1979-101A, 1966-053J, 1979-101A, 1975-118C, 1967-066G, 1975-118C, 1975-118C, 1975-118C, 1982-093A, 1975-123C, 1975-118C, 1975-118C, 1975-117A, 1977-092A, 1982-093A, 1982-093A, 1975-123C, 1975-117A, 1975-117A, 1975-123C, 1982-093A, 1975-117A, 1982-093A, 1975-117A, 1975-117A, 1991-803A, 1991-803A, 1975-117A, 1975-117A, 1975-123C, 1979-087A, 1979-053C, 1982-093A, 1988-040B, 1988-040B, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1975-117A, 1977-092A, 1979-087A, 1975-117A, 1975-117A, 1975-117A, 1975-117A, 1980-018A, 1980-018A, 1982-093A, 1982-093A, 1975-118C, 1982-093A, 1975-118C, 1982-093A, 1979-053C, 1977-092A, 1977-092A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1967-066G, 1975-118C, 1975-117A, 1975-117A, 1982-093A, 1979-087A, 1979-087A, 1975-117A, 1977-092A, 1977-092A, 1979-087A, 1980-018A, 1982-093A, 1975-117A, 1980-018A, 1980-018A, 1975-123C, 1975-123C, 1975-123C, 1982-093A, 1979-053C, 1979-053C, 1979-101A, 1975-117A, 1975-117A, 1985-105A, 1985-105A, 1979-087A, 1979-053C, 1982-093A, 1975-117A, 1979-087A, 1982-093A, 1982-093A, 1975-123C, 1982-093A, 1979-087A, 1979-053C, 1982-093A, 1975-123C, 1991-067A, 1977-092A, 1975-117A, 1982-093A, 1979-087A, 1979-087A, 1982-093A, 1982-093A, 1982-093A, 1979-053C, 1979-087A, 1982-093A, 1985-105A, 1977-092A, 1975-123C, 1979-053C, 1979-053C, 1991-802A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1975-117A, 1975-117A, 1975-117A, 1975-117A, 1975-117A, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1975-117A, 1982-093A, 1979-087A, 1979-087A, 1975-117A, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1985-056B, 1975-118C, 1979-053C, 1982-093A, 1975-117A, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1979-053C, 1982-093A, 1982-093A, 1979-053C, 1979-053C, 1975-117A, 1979-053C, 1979-087A, 1982-093A, 1982-093A, 1979-053C, 1979-053C, 1975-117A, 1979-053C, 1977-092A, 1979-087A, 1979-087A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1975-117A, 1979-101A, 1975-123C, 1975-123C, 1975-117A, 1967-066G, 1982-093A, 1980-018A, 1988-040B, 1979-101A, 1979-053C, 1982-093A, 1977-092A, 1975-117A, 1979-053C, 1977-092A, 1979-101A, 1982-093A, 1979-087A, 1982-093A, 1982-093A, 1979-101A, 1982-093A, 1982-093A, 1975-123C, 1975-123C, 1979-053C, 1979-053C, 1985-105A, 1975-117A, 1975-117A, 1980-018A, 1977-092A, 1975-123C, 1979-053C, 1979-053C, 1979-053C, 1979-087A, 1979-053C, 1982-093A, 1975-123C, 1979-087A, 1980-018A, 1980-018A, 1975-118C, 1975-118C, 1975-118C, 1975-118C, 1975-117A, 2007-003B, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1980-018A, 1979-053C, 1975-123C, 1975-123C, 1982-093A, 1982-093A, 1982-093A, 1982-093A, 1975-117A, 1975-117A, 1979-087A, 1979-087A, 1979-087A, 1979-101A, 1985-105A, 1985-105A, 1975-123C, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1979-101A, 1975-117A, 1979-053C, 1979-087A, 1985-105A, 1979-087A, 1982-093A, 1980-018A, 1979-087A, 1979-087A, 1979-053C, 1979-087A, 1975-117A, 1982-093A, 1975-123C, 1982-093A, 1979-087A, 1979-053C, 1985-105A, 1973-040B, 1979-053C, 1967-066G, 1985-105A, 1985-105A, 1982-093A, 1982-093A, 1979-053C, 1975-118C, 1975-117A, 1979-087A, 1975-118C, 1979-087A, 1975-118C, 1979-087A, 1979-087A, 1985-056B, 1985-056B, 1975-117A, 1975-123C, 1975-123C, 1975-123C, 1975-123C, 1980-018A, 2007-003B, 1975-117A, 1979-101A, 1982-093A, 1991-067A, 1985-105A, 1979-087A, 1985-105A, 1979-053C, 1975-123C, 1975-123C, 1979-087A, 1975-117A, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1991-075B, 1992-072B, 1991-075B, 1992-072B, 1992-072B, 1988-040B, 1985-056B, 1985-056B, 1985-056B, 1991-075B, 1992-072B, 1988-040B, 1991-075B, 1985-056B, 1992-072B, 1992-072B, 1985-056B, 1992-072B, 1992-072B, 1988-040B, 1992-072B, 1985-056B, 1988-040B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1985-056B, 1967-066G, 1966-053J, 1966-053J, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1985-056B, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1979-101A, 1967-066G, 1985-056B, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1967-066G, 1975-117A, 1967-066G, 1975-117A, 1967-066G, 1967-066G, 1985-056B, 1967-066G, 1966-053J, 1967-066G, 1967-066G, 1979-101A, 1979-101A, 1988-040B, 1967-066G, 1991-803A, 1967-066G, 1979-101A, 1967-066G, 1967-066G, 1979-101A, 1967-066G, 1975-117A, 1967-066G, 1979-101A, 1979-101A, 1988-040B, 1967-066G, 1979-101A, 1979-101A, 1967-066G, 1991-803A, 1967-066G, 1985-056B, 1979-053C, 1975-117A, 1975-117A, 1975-117A, 1979-101A, 1967-066G, 1979-101A, 1979-101A, 1982-093A, 1975-123C, 1980-018A, 1975-123C, 1967-066G, 1980-018A, 1982-093A, 1975-117A, 1982-093A, 1979-101A, 1979-087A, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1977-092A, 1979-087A, 1982-093A, 1982-093A, 1975-117A, 1975-118C, 1975-118C, 1975-117A, 1975-117A, 1977-092A, 1979-053C, 1988-040B, 1979-087A, 1979-087A, 1979-101A, 1982-093A, 1982-093A, 1979-087A, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1982-093A, 1979-087A, 1977-092A, 1977-092A, 1977-092A, 1977-092A, 1982-093A, 1975-123C, 1975-123C, 1982-093A, 1967-066G, 1982-093A, 1979-053C, 1979-101A, 1982-093A, 1982-093A, 1979-087A, 1975-117A, 1975-117A, 1975-117A, 1980-018A, 1980-018A, 1980-018A, 1975-123C, 1975-117A, 1979-101A, 1979-053C, 1982-093A, 1975-117A, 1985-105A, 1979-053C, 1975-123C, 1975-123C, 1975-123C, 1979-087A, 1975-117A, 1982-093A, 1975-117A, 1973-040B, 1973-040B, 1973-040B, 1977-092A, 1977-092A, 1975-117A, 1982-093A, 1991-067A, 1975-117A, 1982-093A, 1982-093A, 1982-093A, 1979-087A, 1982-093A, 1979-053C, 1977-092A, 1977-092A, 1975-123C, 1975-123C, 1975-123C, 1977-092A, 1977-092A, 1979-053C, 1975-123C, 1975-117A, 1979-053C, 1975-117A, 1979-087A, 1982-093A, 1979-087A, 1982-093A, 1979-053C, 1975-117A, 1975-117A, 1975-123C, 1988-040B, 1982-093A, 1975-118C, 1979-087A, 1979-053C, 1975-117A, 1975-117A, 1982-093A, 1985-105A, 1985-056B, 1979-053C, 1979-087A, 1975-117A, 1979-101A, 1982-093A, 1982-093A, 1979-087A, 1979-053C, 1979-087A, 1982-093A, 1975-117A, 1979-087A, 1979-087A, 1975-117A, 1982-093A, 1975-117A, 1975-118C, 1977-092A, 1977-092A, 1977-092A, 1979-087A, 1975-117A, 1980-018A, 1979-053C, 1973-040B, 1979-053C, 1975-123C, 1975-123C, 1979-087A, 1979-087A, 1975-117A, 1975-123C, 1982-093A, 1979-101A, 1982-093A, 1979-101A, 1977-092A, 1979-053C, 1979-053C, 1977-092A, 1979-101A, 1975-117A, 1967-066G, 1979-087A, 1979-053C, 1979-053C, 1979-087A, 1975-117A, 1975-117A, 1975-117A, 1975-117A, 1979-101A, 1967-066G, 1979-087A, 1975-123C, 1967-066G, 1975-117A, 1975-117A, 1982-093A, 1979-053C, 1982-093A, 1967-066G, 1979-053C, 1975-117A, 1979-101A, 1982-093A, 1982-093A, 1975-117A, 1985-105A, 1975-117A, 1975-117A, 1979-053C, 1975-117A, 1975-117A, 1975-123C, 1979-087A, 1977-092A, 1975-123C, 1975-117A, 1975-117A, 1982-093A, 1979-053C, 2007-003B, 2007-003B, 1979-101A, 1967-066G, 1975-118C, 1985-105A, 1967-066G, 1980-057A, 1980-057A, 1980-057A, 1980-057A, 1979-077A, 1979-077A, 1980-057A, 1980-057A, 1979-077A, 1979-077A, 1980-057A, 1980-057A, 1980-057A]


22032 	ARIANE 44L R/B	1992-041C	ROCKET BODY	FR	1992-07-09	FRGUI	2021-03-16	306.97	6.99	17167	224	LARGE	TLE | OMM
21766 	ARIANE 44L R/B	1991-075B	ROCKET BODY	FR	1991-10-29	FRGUI	2014-10-20	91.09	7.22	527	130	LARGE	TLE | OMM
22206 	ARIANE 42P+ R/B	1992-072B	ROCKET BODY	FR	1992-10-28	FRGUI	1994-08-26	93.19	7.07	751	111	LARGE	TLE | OMM
15876 	ARIANE 1 R/B	1985-056B	ROCKET BODY	FR	1985-07-02	FRGUI		401.52	9.02	22825	477	LARGE	TLE | OMM
19122 	ARIANE 2 R/B	1988-040B	ROCKET BODY	FR	1988-05-17	FRGUI		605.62	7.45	34141	526	LARGE	TLE | OMM
2868 	TITAN 3C TRANSTAGE R/B	1967-066G	ROCKET BODY	US	1967-07-01	AFETR		1322.21	1.61	33859	33195	LARGE	TLE | OMM
8476 	SATCOM 1	1975-117A	PAYLOAD	US	1975-12-13	AFETR		1445.78	8.69	36041	35911	LARGE	TLE | OMM
49960 	SATCOM 3 DEB	1979-101AE	DEBRIS	US	1979-12-07	AFETR		770.11	8.77	35875	7031		TLE | OMM
49959 	SATCOM 3 DEB	1979-101AD	DEBRIS	US	1979-12-07	AFETR		797.03	6.52	35804	8392	SMALL	TLE | OMM
49958 	SATCOM 3 DEB	1979-101AC	DEBRIS	US	1979-12-07	AFETR		832.60	11.26	36202	9675	SMALL	TLE | OMM
49957 	SATCOM 3 DEB	1979-101AB	DEBRIS	US	1979-12-07	AFETR		791.17	8.18	35716	8201		TLE | OMM
49956 	SATCOM 3 DEB	1979-101AA	DEBRIS	US	1979-12-07	AFETR		804.07	7.13	35743	8788	MEDIUM	TLE | OMM
11635 	SATCOM 3	1979-101A	PAYLOAD	US	1979-12-07	AFETR		788.93	8.40	35452	8357	LARGE	TLE | OMM
8516 	TITAN 3C TRANSTAGE R/B	1975-118C	ROCKET BODY	US	1975-12-14	AFETR		1432.35	3.11	35768	35658	LARGE	TLE | OMM
11436 	TITAN 3C TRANSTAGE R/B	1979-053C	ROCKET BODY	US	1979-06-10	AFETR		1448.06	7.64	36276	35765		TLE | OMM		 
8515 	SL-12 R/B(1)	1975-123C	ROCKET BODY	CIS	1975-12-22	TTMTR	1975-12-25	87.44	51.47	150	145		TLE | OMM
11715 	AYAME 2 (ECS-2)	1980-018A	PAYLOAD	JPN	1980-02-22	TNSTA		820.54	9.20	35408	9902	MEDIUM	TLE | OMM
11561 	EKRAN 4	1979-087A	PAYLOAD	CIS	1979-10-03	TTMTR		1435.70	6.41	35833	35724	MEDIUM	TLE | OMM
10365 	EKRAN 2	1977-092A	PAYLOAD	CIS	1977-09-20	TTMTR		1435.36	5.08	35908	35637	LARGE	TLE | OMM
13554 	EKRAN 9	1982-093A	PAYLOAD	CIS	1982-09-16	TTMTR		1435.14	7.82	35877	35659	MEDIUM	TLE | OMM
21726 	ANIK E1	1991-067A	PAYLOAD	CA	1991-09-26	FRGUI		1451.98	13.85	36133	36061	LARGE	TLE | OMM
2222 	TITAN 3C TRANSTAGE R/B	1966-053J	ROCKET BODY	US	1966-06-16	AFETR		1349.31	7.76	34731	33410	LARGE	TLE | OMM
16235 	COSMOS 1701	1985-105A	PAYLOAD	CIS	1985-11-09	PKMTR	2001-05-11	91.92	62.33	656	82	LARGE	TLE | OMM
11871 	COSMOS 1191	1980-057A	PAYLOAD	CIS	1980-07-02	PKMTR		716.54	65.71	37873	2421	LARGE	TLE | OMM


In [ ]:
import requests
from urllib.parse import quote_plus


username = "fiona.paerli@students.unibe.ch"
password = "tlefiona2024aiub"

session = requests.Session()
login_url = "https://www.space-track.org/ajaxauth/login"
login_payload = {"identity": username, "password": password}

r = session.post(login_url, data=login_payload)
print("Login status code:", r.status_code)
print("Login response:", r.text)
print(session.cookies.get_dict())


if r.status_code == 200:
    where = quote_plus("INTLDES='1992-041C'")
    where = quote_plus("1=1")
    url = f"https://www.space-track.org/basicspacedata/query/class/satcat/format/json/WHERE/{where}"
    
    r2 = session.get(url)
    print("Query status code:", r2.status_code)
    print("Query response:", r2.text)
else:
    print("Login failed, cannot query data")

#try this: https://stackoverflow.com/questions/76463658/how-to-generate-data-from-space-track-website-using-api


Login status code: 200
Login response: ""
{'chocolatechip': 'ndul2atppdft8sp72cksdk74rjsavqru', 'GCLB': 'COOUo7f7jOrQ3wEQAw'}
Query status code: 500
Query response: <!DOCTYPE html>
<html lang="en-US">
<head>
    <title>Space-Track.Org</title>
    <meta charset="utf-8">
    <meta name="viewport"    content="width=device-width, initial-scale=1.0">
    <meta name="description" content="Space-Track.Org">
    <meta name="keywords"    content="Space, TLE, orbit, satellites, space objects">
    <meta name="author"      content="SAIC admin@space-track.org">
    <meta name="googlebot"   content="noindex">
    <meta name="apple-mobile-web-app-title" content="Space-Track">
    <meta name="application-name" content="Space-Track">
    <meta name="theme-color" content="#ffffff">

    <link rel="apple-touch-icon" sizes="180x180" href="https://www.space-track.org/assets/ico/prod/apple-touch-icon.png">
    <link rel="icon" type="image/png" sizes="32x32" href="https://www.space-track.org/assets/ico/prod

In [18]:
import requests



username = "fiona.paerli@students.unibe.ch"
password = "tlefiona2024aiub"

# Base URL
base_url = "https://www.space-track.org"

# Endpoints
satcat_endpoint = "/basicspacedata/query/class/satcat/INTDES/{satellite_intdes}/orderby/SATNAME/format/json"
tle_endpoint = "/basicspacedata/query/class/tle_latest/NORAD_CAT_ID/{norad_cat_id}/orderby/EPOCH desc/limit/1/format/tle"

# Function to fetch SATCAT data
def fetch_satcat_data(satellite_intdes):
    url = base_url + satcat_endpoint.replace("{satellite_intdes}", satellite_intdes)
    response = requests.get(url, auth=(username, password))

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("❌ Failed to fetch SATCAT data. Status code:", response.status_code)
        print(response.text)
        return None

# Function to fetch TLE data
def fetch_tle_data(norad_cat_id):
    url = base_url + tle_endpoint.replace("{norad_cat_id}", norad_cat_id)
    response = requests.get(url, auth=(username, password))

    if response.status_code == 200:
        return response.text
    else:
        print("❌ Failed to fetch TLE data. Status code:", response.status_code)
        print(response.text)
        return None

# ✅ Example: ISS by International Designator
# INTDES for the ISS is "1998-067A"
satellite_intdes = "1998-067A"
satcat_data = fetch_satcat_data(satellite_intdes)
if satcat_data:
    print("\n✅ SATCAT data example:")
    for item in satcat_data:
        print(item)
        break  # just print first entry

# ✅ Example: ISS by NORAD ID 25544
norad_cat_id = "25544"
tle_data = fetch_tle_data(norad_cat_id)
if tle_data:
    print("\n✅ Latest TLE for NORAD ID", norad_cat_id, ":\n")
    print(tle_data)


❌ Failed to fetch SATCAT data. Status code: 401
{"error":"You must be logged in to complete this action"}
❌ Failed to fetch TLE data. Status code: 204



In [5]:
import requests
from requests.exceptions import HTTPError

#https://realpython.com/python-requests/

response = requests.get("https://api.github.com") #make a GET request

#work with status codes
if response:
    print("Success!", response.status_code)
else:
    raise Exception(f"Non-success status code: {response.status_code}")


URLS = ["https://api.github.com", "https://api.github.com/invalid"]

for url in URLS:
    try:
        response = requests.get(url)
        response.raise_for_status()
    except HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"Other error occurred: {err}")
    else:
        print("Success!")

#access the response content
response = requests.get("https://api.github.com") 
print(response.content, type(response.content))
response.encoding = "utf-8"#define the encoding, when not specified, will try based on response's header
print(response.text, type(response.text))#returns a string

print(response.json(), type(response.json()))#returns a dictionary 
response_dict = response.json()#playing around with dictionary
print(response_dict["emojis_url"])

#view response header
response = requests.get("https://api.github.com")
print(response.headers) #returns a dictionary-like object
print( response.headers["Content-Type"])#case-insensitive, do not worry about capitalization




#add query string parameters
response = requests.get(
    "https://api.github.com/search/repositories",
    params={"q": "language:python", "sort": "stars", "order": "desc"}, #you can either pass the params as a dictionary or also as a list of tuples or as bytes
)

json_response = response.json()
popular_repositories = json_response["items"]
for repo in popular_repositories[:3]:
    print(f"Name: {repo['name']}")
    print(f"Description: {repo['description']}")
    print(f"Stars: {repo['stargazers_count']}\n")

#customize request headers
response = requests.get(
    "https://api.github.com/search/repositories",
    params={"q": '"real python"'},
    headers={"Accept": "application/vnd.github.text-match+json"}, 
    # The Accept header tells the server what content types your application can handle. 
    # In this case, since you’re expecting the matching search terms to be highlighted, you’re using the header value application/vnd.github.text-match+json. 
    # This is a proprietary GitHub Accept header where the content is a special JSON format.
)

json_response = response.json()
first_repository = json_response["items"][0]
print(first_repository["text_matches"][0]["matches"])

#use other http methods
#use hhtpbin.opg to test these methods, service for testing/training 
requests.get("https://httpbin.org/get") #this is a shortcut for: requests.request("GET", "https://httpbin.org/get"), same for other functions
requests.post("https://httpbin.org/post", data={"key": "value"})
requests.put("https://httpbin.org/put", data={"key": "value"})

response = requests.delete("https://httpbin.org/delete")#we can inspect the respones as before
json_response = response.json()
print(json_response["args"])

requests.head("https://httpbin.org/get")
requests.patch("https://httpbin.org/patch", data={"key": "value"})
requests.options("https://httpbin.org/get")

Exception: Non-success status code: 403

# Explanation of all the methods
GET: used to retrieve data from the server  
POST: send new data to the server  
PUT: upload or replace a resource (replaces the whole thing)  
DELETE: remove a resource  
HEAD: same as get but only retrieves headers, not body
PATCH: partially updata a resource 
OPTIONS: ask the server which HTTP methods are allowed for a URL

In [8]:
#send request data
import requests

#you can send the data as a dictionary, list of tuples or as json
requests.post("https://httpbin.org/post", data={"key": "value"})
requests.post("https://httpbin.org/post", data=[("key", "value")])
response = requests.post("https://httpbin.org/post", json={"key": "value"})
json_response = response.json()
json_response["data"]

json_response["headers"]["Content-Type"]

#inspect the prepared request
#When you make a request, the Requests library prepares the request before actually sending it to the destination server. 
#Request preparation includes things like validating headers and serializing JSON content.
response = requests.post("https://httpbin.org/post", json={"key":"value"})
print(response.request)
print(response.request.headers["Content-Type"])
print(response.request.url)
print(response.request.body)

<PreparedRequest [POST]>
application/json
https://httpbin.org/post
b'{"key": "value"}'


In [13]:
#use authentication
import requests

response = requests.get(
    "https://httpbin.org/basic-auth/user/passwd",
    auth=("user", "passwd")
)

print(response.status_code)

print(response.request.headers["Authorization"])

#when you try to make a request without authification, you get error 401
requests.get("https://api.github.com/user")

#github API: first need to generate a personal access token
token = "<YOUR_GITHUB_PA_TOKEN>"
response = requests.get(
    "https://api.github.com/user",
    auth=("", token)
)
print(response.status_code)
print(response.request.headers["Authorization"])

200
Basic dXNlcjpwYXNzd2Q=
401
Basic OjxZT1VSX0dJVEhVQl9QQV9UT0tFTj4=


In [17]:
#make your own authetification mechanism

from requests.auth import AuthBase

class TokenAuth(AuthBase):
    """Implements a token authentication scheme."""

    def __init__(self, token):
        self.token = token

    def __call__(self, request):
        """Attach an API token to the Authorization header."""
        request.headers["Authorization"] = f"Bearer {self.token}"
        return request
    
token = "<YOUR_GITHUB_PA_TOKEN>"
response = requests.get(
    "https://api.github.com/user",
    auth=TokenAuth(token)
)

print(response.status_code)

response.request.headers["Authorization"]
#more intuitive and reusable way to work with token-based authentication schemes

401


'Bearer <YOUR_GITHUB_PA_TOKEN>'